# Training and Testing Neural Networks using the New Optimizers

### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Custom models and optimizers

In [ ]:
from models import SimpleCNN, SimpleMLP     # defined in models.py
from new_optimizers import MomentumSGD_Strong_Wolfe, Adam_Strong_Wolfe, TrustRegionCauchy   # defined in new_optimizers.py

## Task 1: Digit Classification on MNIST Using CNNs

### Downloading the dataset and normalizing

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # mean and std for MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

### Preparing PyTorch dataLoaders and initializing the model (CNN)

In [ ]:
torch.manual_seed(42)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleConvNN().to(device)

### Listing new optimizers and benchmark optimizers

In [ ]:
# Benchmarks
optimizer1 = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.001)

# Custom optimizers
optimizer3 = MomentumSGD_Strong_Wolfe(model.parameters(), alpha1=0.001)
optimizer4 = Adam_Strong_Wolfe(model.parameters(), alpha1=0.01, betas=(0.9, 0.999))
optimizer5 = TrustRegionCauchy(model.parameters())

### Defining train and test functions

In [ ]:
criterion = nn.CrossEntropyLoss()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        def closure(backward=True):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            if backward:
                loss.backward()
            return loss
        
        loss = closure()
        optimizer.step(closure)
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({100. * correct / len(test_loader.dataset):.0f}%)\n')

### Training and plotting losses over epochs

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)